In [1]:
from netCDF4 import Dataset
from netCDFfunc.utility import get_data_sequence, get_data_A, get_data_by_date, save_img, show_img, masking

import numpy as np
import matplotlib.pyplot as plt

from collections import Counter

In [2]:
base_dir = '/Volumes/T7/AVHRR_OI_SST/v2.1/'

for year in range(1982, 2021):
    value = get_data_sequence(base_dir = base_dir,
                              get_data_func = get_data_A,
                              var_name = 'sst',
                              start_date = (year,1,1),
                              end_date = (year,12,31))
    ice = get_data_sequence(base_dir = base_dir,
                              get_data_func = get_data_A,
                              var_name = 'ice',
                              start_date = (year,1,1),
                              end_date = (year,12,31),
                            is_mask=True)
    
    result = []
    for v, i in zip(value, ice):
        iced_pixel = np.ma.array(v, mask=i)
        result.append(iced_pixel.filled(fill_value=np.nan))
    
    distribution = np.array(result).flatten()
    np.place(distribution, distribution == -999, np.nan)
    min_value = np.nanmin(distribution)
    max_value = np.nanmax(distribution)
    plt.hist(distribution, bins=100)
    plt.savefig(f'/Volumes/T7/intermediate output/iced_pixel_values/{year}_min=({min_value:.2f})&max=({max_value:.2f}).jpg', dpi=100)
    plt.close()

In [47]:
def monthly_ice_mask(month):
    sst = get_data_by_date(base_dir = base_dir,
                            get_data_func = get_data_A,
                            var_name = 'sst',
                            start_date = (1982,1,1),
                            end_date = (2021,12,31),
                            specific_date=(month,1),
                            is_mask=False)

    ice = get_data_by_date(base_dir = base_dir,
                            get_data_func = get_data_A,
                            var_name = 'ice',
                            start_date = (1982,1,1),
                            end_date = (2021,12,31),
                            specific_date=(month,1),
                            is_mask=True)

    ice_sst_arr = masking(np.array(sst[(month,1)]), np.array(ice[(month,1)]), fill_value=np.nan)
    
    return ice_sst_arr

year = 1982
for ice_sst in ice_sst_arr :
    save_img(ice_sst, f'/Volumes/T7/other_data/ice_img/7/{year}.jpg')
    year += 1

In [104]:
def get_ice_dist(month, delete_normal=True):
    ice = get_data_by_date(base_dir = base_dir,
                            get_data_func = get_data_A,
                            var_name = 'ice',
                            start_date = (1982,1,1),
                            end_date = (2021,12,31),
                            specific_date=(month,1),
                            is_mask=True)
    ice_dist = np.sum(np.invert(np.array(ice[(month,1)])), axis=0).flatten()
    normal_per = Counter(ice_dist)[0] / len(ice_dist) * 100 + Counter(ice_dist)[40] / len(ice_dist) * 100
    anormal_per = round(100 - normal_per,1)
    
    if delete_normal == True :
        ice_dist = np.delete(ice_dist, np.where(ice_dist==0))
        ice_dist = np.delete(ice_dist, np.where(ice_dist==40))
        bins = 39
    else :
        bins = 41

    plt.hist(ice_dist, bins=bins)
    plt.savefig(f'/Volumes/T7/intermediate output/monthly_ice_pixel_anormally_distribution/{month}(anormal_data = {anormal_per}%).jpg')
    plt.close()
    
    return ice_dist

In [105]:
for i in range(1,13):
    ice_dist = get_ice_dist(i, delete_normal=True)

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

...

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

In [86]:
round(Counter(ice_dist)[0] / len(ice_dist) * 100 + Counter(ice_dist)[40] / len(ice_dist) * 100, 1)

92.9

In [106]:
Counter(ice_dist)

Counter({5: 2152,
         4: 1869,
         17: 4736,
         37: 3918,
         38: 4557,
         39: 8468,
         35: 1906,
         33: 1488,
         31: 1383,
         32: 1337,
         30: 1386,
         29: 1306,
         34: 1655,
         16: 1270,
         28: 1170,
         36: 2474,
         14: 1168,
         23: 1155,
         25: 859,
         21: 1054,
         11: 1124,
         9: 1099,
         8: 1256,
         7: 1429,
         12: 1033,
         27: 1001,
         26: 909,
         24: 908,
         22: 1020,
         20: 956,
         18: 958,
         19: 1031,
         13: 1117,
         10: 1036,
         15: 1146,
         3: 1984,
         6: 1434,
         2: 3484,
         1: 5114})